In [1]:
from langchain_community.tools import WikipediaQueryRun

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper

In [22]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [23]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/sanimpandey/Desktop/tech/LLMS/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [24]:
wiki.name

'wikipedia'

In [6]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")

In [8]:
loader

In [10]:
data=loader.load()

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
docs=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(data)

In [13]:
vectordb=Chroma.from_documents(docs,OllamaEmbeddings(model="llama3"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x12162fb30>)

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search for information about Langsmith. For any questions about Langsmith you must use this tool.")

In [18]:
retriever_tool.name

'langsmith_search'

In [19]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [20]:
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [21]:
arxiv.name

'arxiv'

In [25]:
tools=[wiki,retriever_tool,arxiv]

In [26]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/sanimpandey/Desktop/tech/LLMS/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about Langsmith. For any questions about Langsmith you must use this tool.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x116da9ee0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x12162fb30>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x116daa3e0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_commu

In [27]:
from langchain_community.llms import Ollama
#load ollama llama3 llm model
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

In [39]:
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format

In [40]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [41]:
agent = create_structured_chat_agent(llm, tools, prompt)

In [42]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [51]:
agent_executor.invoke({"input":"What is the capital of Nepal?"})



> Entering new AgentExecutor chain...
Thought: The question asks for the capital of Nepal.
Action:
```
{
  "action": "wikipedia",
  "action_input": {"query": {"title": "Kathmandu", "description": "capital of Nepal"}}
}
``

ValidationError: 1 validation error for WikipediaQueryInput
query
  str type expected (type=type_error.str)